#### IMPORTO LIBRERIAS ####

In [25]:
import pandas as pd
import numpy as np
from sklearn import linear_model

#configuro notebook para mostrar varios salidas en la misma celda
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#amplio el output de columnas a 500
pd.set_option('display.max_columns', 500)

!pwd
!ls -l

/Users/mac/Documents/MASTER_DATA_SCIENCE/tfm
total 856
-rw-r--r--   1 mac  staff  104803 15 abr 11:08 TFM_DGT-pruebas.ipynb
-rw-r--r--   1 mac  staff  331497 30 may 22:56 TFM_DGT.ipynb
drwxr-xr-x  14 mac  staff     476 30 may 11:10 cleaned
-rw-r--r--   1 mac  staff       0 23 may 22:42 gg
drwxr-xr-x   5 mac  staff     170 30 mar 23:28 metadata
drwxr-xr-x  11 mac  staff     374 30 mar 23:28 raw
drwxr-xr-x  14 mac  staff     476 11 may 17:18 raw_zip


### CONSERVO LOS DATOS ORIGINALES EN LA CARPETA RAW. LOS FICHEROS DE TRABAJO ESTAN EN LA CARPETA CLEANED

In [26]:
!ls -l
!cp -r raw/* cleaned/

total 856
-rw-r--r--   1 mac  staff  104803 15 abr 11:08 TFM_DGT-pruebas.ipynb
-rw-r--r--   1 mac  staff  331497 30 may 22:56 TFM_DGT.ipynb
drwxr-xr-x  14 mac  staff     476 30 may 11:10 cleaned
-rw-r--r--   1 mac  staff       0 23 may 22:42 gg
drwxr-xr-x   5 mac  staff     170 30 mar 23:28 metadata
drwxr-xr-x  11 mac  staff     374 30 mar 23:28 raw
drwxr-xr-x  14 mac  staff     476 11 may 17:18 raw_zip


### GUARDO UN LISTADO POR TIPOLOGIA DE FICHERO, PARA POSTERIORMENTE ALMACENARLO EN UN DICCIONARIO ###

In [27]:
accvict=!ls ./cleaned/MICRODATOS_ACC_VICT_20*/TABLA_ACCVICT_*.csv
pers=!ls ./cleaned/MICRODATOS_ACC_VICT_20*/TABLA_PERS_*.csv
vehic=!ls ./cleaned/MICRODATOS_ACC_VICT_20*/TABLA_VEHIC_*.csv
dic_all_files={'Accvict':accvict,'Pers':pers,'Vehic':vehic}
len(dic_all_files)

3

#### RUTAS DE TODOS LOS FICHEROS ####

In [28]:
dic_all_files

{'Accvict': ['./cleaned/MICRODATOS_ACC_VICT_2008/TABLA_ACCVICT_24H_2008.csv',
  './cleaned/MICRODATOS_ACC_VICT_2009/TABLA_ACCVICT_24H_2009.csv',
  './cleaned/MICRODATOS_ACC_VICT_2010/TABLA_ACCVICT_24H_2010.csv',
  './cleaned/MICRODATOS_ACC_VICT_2011/TABLA_ACCVICT_2011.csv',
  './cleaned/MICRODATOS_ACC_VICT_2012/TABLA_ACCVICT_2012.csv',
  './cleaned/MICRODATOS_ACC_VICT_2013/TABLA_ACCVICT_2013.csv',
  './cleaned/MICRODATOS_ACC_VICT_2014/TABLA_ACCVICT_2014.csv',
  './cleaned/MICRODATOS_ACC_VICT_2015/TABLA_ACCVICT_2015.csv'],
 'Pers': ['./cleaned/MICRODATOS_ACC_VICT_2008/TABLA_PERS_24H_2008.csv',
  './cleaned/MICRODATOS_ACC_VICT_2009/TABLA_PERS_24H_2009.csv',
  './cleaned/MICRODATOS_ACC_VICT_2010/TABLA_PERS_24H_2010.csv',
  './cleaned/MICRODATOS_ACC_VICT_2011/TABLA_PERS_2011.csv',
  './cleaned/MICRODATOS_ACC_VICT_2012/TABLA_PERS_2012.csv',
  './cleaned/MICRODATOS_ACC_VICT_2013/TABLA_PERS_2013.csv',
  './cleaned/MICRODATOS_ACC_VICT_2014/TABLA_PERS_2014.csv',
  './cleaned/MICRODATOS_ACC_VICT

### Ahora que los datos estan trazados, procedemos a realizar una observacion preliminar para comprobar si hay homgeneidad en los datos, y si disponemos de la misma informacion en cada data set, ya que segun la DGT, las extructuras de los datos cambian a partir del 2011 en hasta el 2015.

### Por los que tenemos dos grupos de datos:

### Tipo 1) 2008-2010
### Tipo 2) 2011-2015

### Recorro cada clave - para obtener las cabeceras y poder comparar que campos tengo en cada año por tipo de fichero, e introduzco los data frames en una lista.

In [29]:
dic_all_files.keys()

dict_keys(['Accvict', 'Pers', 'Vehic'])

In [30]:
#Creo una lista con las cabeceras de los ficheros
Headers_List=[]
for key in dic_all_files.keys():
    df_2008=pd.read_csv(dic_all_files[key][0],delimiter=';',nrows=0,encoding="ISO-8859-1")
    df_2008=df_2008.transpose()
    df_2008.reset_index(inplace=True)
    df_2008.columns=[key+'_2008']

    df_2015=pd.read_csv(dic_all_files[key][7],delimiter=';',nrows=0,encoding="ISO-8859-1")
    df_2015=df_2015.transpose()
    df_2015.reset_index(inplace=True)
    df_2015.columns=[key+'_2015']
        
    Headers_List.append(df_2008.merge(df_2015, left_on=key+'_2008', right_on=key+'_2015', how='outer'))

### --------------------------------------------------------------------------------------------------

## REVISION DE LOS TRES TIPOS DE DATA SETS

### SE DISPONE DE TRES TIPOS DE DATASETS:

#### 1. ACCIDENTES
#### 2. PERSONAS
#### 3 .VEHICULOS

#### COMO PRIMER PASO VOY A REVISAR TANTO ESTRUCTURA, PARA DECIDIR QUE VARIABLES MANTENDRE PARA EL ANALISIS POSTERIOR. PROCEDERE A MODIFICAR LOS DATASET DE LA CARPETA "./CLEANED" ANTES DE REALIZAR LA CARGA PARA EN ANALISIS.

### --------------------------------------------------------------------------------------------------

###   1. ACCIDENTES

In [31]:
#Cabecera del fichero de Accidentes_victimas
Header_Accvict=Headers_List[0]
Header_Accvict

,Accvict_2008,Accvict_2015
0,ID_ACCIDENTE,ID_ACCIDENTE
1,ANIO,ANIO
2,MES,MES
3,HORA,HORA
4,DIASEMANA,DIASEMANA
5,PROVINCIA,PROVINCIA
6,COMUNIDAD_AUTONOMA,COMUNIDAD_AUTONOMA
7,ISLA,ISLA
8,MUNICIPIO,MUNICIPIO
9,TOT_VICTIMAS,TOT_VICTIMAS


In [32]:
#mantengo solo las columnas que son iguales en los dos tipos de dataset

Header_Accvict=Header_Accvict[Header_Accvict['Accvict_2008']==Header_Accvict['Accvict_2015']]
Header_Accvict=Header_Accvict['Accvict_2008'].tolist()
dic_all_headers={'Accvict':Header_Accvict}
dic_all_headers

#Genero nuevo DF
df_all_years=pd.DataFrame(index=None, columns=dic_all_headers['Accvict'])

#Cargo cada csv y concateno
for file in dic_all_files['Accvict']:
    df_vict=pd.read_csv(file,delimiter=';',usecols=dic_all_headers['Accvict'],encoding="ISO-8859-1")
    #relleno los nulos con 999
    df_vict.fillna(999,inplace=True)
    df_all_years=pd.concat([df_all_years,df_vict],ignore_index=True)
    
#muestreo los datos
df_all_years.head(3)
df_all_years.tail(3)
df_all_years.sample(3)

#De las columnas del data frame final, que contiene todos los años, 
#examino los distintos valores de las columnas, para tratar de decidir 
#que campos conservare para el analisis.
for col in df_all_years.columns:
    #no saco los distinos valores para los id_accidente
    if col !='ID_ACCIDENTE':
        df_all_years[col].value_counts(dropna=False)
        
#Una vez examindas los resultado anteriores, decido descartar los siguientes campos:
df_vict.drop(['ISLA','MUNICIPIO','TIPO_INTERSEC','ACERAS','VISIBILIDAD_RESTRINGIDA','CARRETERA','TRAZADO_NO_INTERSEC'], axis=1, inplace=True)

#Actualizo mi diccionario con las cabeceras que utilizare en el proceso de carga
Header_Accvict=df_vict.columns.tolist()
dic_all_headers={'Accvict':Header_Accvict}
dic_all_headers
#En el campo factores atmosfericos, como lo considero relevante
#y tengo 4787 valores desconocidos, englobo estos registros dentro de la categoria otros(9)

for file in dic_all_files['Accvict']:
    df_acc=pd.read_csv(file,delimiter=";",usecols=dic_all_headers['Accvict'],encoding="ISO-8859-1")
    df_acc.fillna(999,inplace=True)
    #factores atmosfericos desconocidos,tal y como cometo arriba van a otros
    df_acc.ix[(df_vict['FACTORES_ATMOSFERICOS']==999), 'FACTORES_ATMOSFERICOS'] =9
    df_acc.to_csv(file,index=False,sep=";",encoding="ISO-8859-1")
    print("MODFICADO ----- "+file)
    
print("CREADA VISTA CON DATOS DE ACCIDENTES Y VISTIMAS DE 2008 A 2015")
df_all_years.to_csv("./cleaned/view_accvict_08_15.csv",index=False,sep=";",encoding="ISO-8859-1")

########   FIN DEL PROCESAMIENTO DE LOS DATASET DE ACCIDENTES #########



{'Accvict': ['ID_ACCIDENTE',
  'ANIO',
  'MES',
  'HORA',
  'DIASEMANA',
  'PROVINCIA',
  'COMUNIDAD_AUTONOMA',
  'ISLA',
  'MUNICIPIO',
  'TOT_VICTIMAS',
  'TOT_MUERTOS',
  'TOT_HERIDOS_GRAVES',
  'TOT_HERIDOS_LEVES',
  'TOT_VEHICULOS_IMPLICADOS',
  'ZONA',
  'ZONA_AGRUPADA',
  'CARRETERA',
  'RED_CARRETERA',
  'TIPO_VIA',
  'TRAZADO_NO_INTERSEC',
  'TIPO_INTERSEC',
  'SUPERFICIE_CALZADA',
  'LUMINOSIDAD',
  'FACTORES_ATMOSFERICOS',
  'VISIBILIDAD_RESTRINGIDA',
  'ACERAS',
  'TIPO_ACCIDENTE']}

,ID_ACCIDENTE,ANIO,MES,HORA,DIASEMANA,PROVINCIA,COMUNIDAD_AUTONOMA,ISLA,MUNICIPIO,TOT_VICTIMAS,TOT_MUERTOS,TOT_HERIDOS_GRAVES,TOT_HERIDOS_LEVES,TOT_VEHICULOS_IMPLICADOS,ZONA,ZONA_AGRUPADA,CARRETERA,RED_CARRETERA,TIPO_VIA,TRAZADO_NO_INTERSEC,TIPO_INTERSEC,SUPERFICIE_CALZADA,LUMINOSIDAD,FACTORES_ATMOSFERICOS,VISIBILIDAD_RESTRINGIDA,ACERAS,TIPO_ACCIDENTE
0,2.00848e+12,2008.0,4.0,7.0,5.0,48.0,17.0,999.0,999.0,4.0,0.0,0.0,4.0,1.0,1.0,1.0,BI-631,3.0,2.0,1.0,999.0,2.0,3.0,1.0,8.0,N,71.0
1,2.00825e+12,2008.0,3.0,14.0,7.0,25.0,9.0,999.0,999.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,C-12,2.0,5.0,2.0,999.0,1.0,1.0,9.0,8.0,N,22.0
2,2.00843e+12,2008.0,10.0,12.0,7.0,43.0,9.0,999.0,999.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,TV-7331,4.0,5.0,1.0,999.0,1.0,1.0,1.0,0.0,N,68.0


,ID_ACCIDENTE,ANIO,MES,HORA,DIASEMANA,PROVINCIA,COMUNIDAD_AUTONOMA,ISLA,MUNICIPIO,TOT_VICTIMAS,TOT_MUERTOS,TOT_HERIDOS_GRAVES,TOT_HERIDOS_LEVES,TOT_VEHICULOS_IMPLICADOS,ZONA,ZONA_AGRUPADA,CARRETERA,RED_CARRETERA,TIPO_VIA,TRAZADO_NO_INTERSEC,TIPO_INTERSEC,SUPERFICIE_CALZADA,LUMINOSIDAD,FACTORES_ATMOSFERICOS,VISIBILIDAD_RESTRINGIDA,ACERAS,TIPO_ACCIDENTE
711899,2016990000875,2015.0,12.0,11.0,4.0,28.0,13.0,999.0,28079.0,2.0,0.0,0.0,2.0,2.0,2.0,2.0,999,4.0,8.0,999.0,2.0,3.0,1.0,4.0,999.0,998,2.0
711900,2016990000876,2015.0,12.0,13.0,2.0,28.0,13.0,999.0,28079.0,1.0,0.0,0.0,1.0,1.0,2.0,2.0,999,4.0,8.0,1.0,999.0,1.0,1.0,1.0,999.0,999,7.0
711901,2016990001660,2015.0,11.0,15.0,1.0,28.0,13.0,999.0,28079.0,1.0,0.0,0.0,1.0,2.0,2.0,2.0,999,4.0,8.0,999.0,2.0,1.0,1.0,1.0,999.0,998,2.0


,ID_ACCIDENTE,ANIO,MES,HORA,DIASEMANA,PROVINCIA,COMUNIDAD_AUTONOMA,ISLA,MUNICIPIO,TOT_VICTIMAS,TOT_MUERTOS,TOT_HERIDOS_GRAVES,TOT_HERIDOS_LEVES,TOT_VEHICULOS_IMPLICADOS,ZONA,ZONA_AGRUPADA,CARRETERA,RED_CARRETERA,TIPO_VIA,TRAZADO_NO_INTERSEC,TIPO_INTERSEC,SUPERFICIE_CALZADA,LUMINOSIDAD,FACTORES_ATMOSFERICOS,VISIBILIDAD_RESTRINGIDA,ACERAS,TIPO_ACCIDENTE
422218,2012990000846,2012.0,1.0,19.0,5.0,28.0,13.0,999.0,28079.0,1.0,0.0,0.0,1.0,2.0,2.0,2.0,999,4.0,8.0,1.0,999.0,1.0,3.0,1.0,999.0,998,4.0
651127,201515054000267,2015.0,8.0,9.0,2.0,15.0,12.0,999.0,15054.0,1.0,0.0,0.0,1.0,1.0,2.0,2.0,999,4.0,8.0,999.0,999.0,1.0,1.0,1.0,999.0,0,7.0
273493,"2,01107E+12",2011.0,11.0,21.0,4.0,7.0,4.0,71.0,7040.0,1.0,0.0,0.0,1.0,2.0,1.0,1.0,Ma-15,3.0,3.0,999.0,3.0,1.0,3.0,1.0,1.0,998,20.0


2015.0    97756
2008.0    93161
2014.0    91570
2013.0    89519
2009.0    88251
2010.0    85503
2012.0    83115
2011.0    83027
Name: ANIO, dtype: int64

...

999.0    256880
1.0      164537
8.0      151022
0.0       94613
2.0       11739
7.0       11337
3.0       10176
4.0        7755
5.0        2681
6.0        1162
Name: VISIBILIDAD_RESTRINGIDA, dtype: int64

998    385762
N      190486
S       48987
999     35153
0       26955
1       24559
Name: ACERAS, dtype: int64

2.0     89924
4.0     82990
12.0    70554
7.0     55923
14.0    51057
3.0     34237
33.0    29470
20.0    27167
5.0     24207
13.0    22316
19.0    21280
16.0    21037
15.0    19466
71.0    15121
9.0     14116
1.0     13998
6.0     12085
68.0    10606
18.0    10454
11.0    10194
41.0     8905
10.0     7449
66.0     7166
64.0     5987
24.0     5595
56.0     4428
54.0     4363
63.0     4289
62.0     3068
52.0     2917
53.0     2890
61.0     2886
8.0      2387
67.0     2095
21.0     1853
51.0     1760
17.0     1732
65.0     1170
36.0     1091
57.0     1013
22.0      855
55.0      798
58.0      536
31.0      256
32.0       94
35.0       55
23.0       37
34.0       25
Name: TIPO_ACCIDENTE, dtype: int64

{'Accvict': ['ID_ACCIDENTE',
  'ANIO',
  'MES',
  'HORA',
  'DIASEMANA',
  'PROVINCIA',
  'COMUNIDAD_AUTONOMA',
  'TOT_VICTIMAS',
  'TOT_MUERTOS',
  'TOT_HERIDOS_GRAVES',
  'TOT_HERIDOS_LEVES',
  'TOT_VEHICULOS_IMPLICADOS',
  'ZONA',
  'ZONA_AGRUPADA',
  'RED_CARRETERA',
  'TIPO_VIA',
  'SUPERFICIE_CALZADA',
  'LUMINOSIDAD',
  'FACTORES_ATMOSFERICOS',
  'TIPO_ACCIDENTE']}

MODFICADO ----- ./cleaned/MICRODATOS_ACC_VICT_2008/TABLA_ACCVICT_24H_2008.csv
MODFICADO ----- ./cleaned/MICRODATOS_ACC_VICT_2009/TABLA_ACCVICT_24H_2009.csv
MODFICADO ----- ./cleaned/MICRODATOS_ACC_VICT_2010/TABLA_ACCVICT_24H_2010.csv
MODFICADO ----- ./cleaned/MICRODATOS_ACC_VICT_2011/TABLA_ACCVICT_2011.csv
MODFICADO ----- ./cleaned/MICRODATOS_ACC_VICT_2012/TABLA_ACCVICT_2012.csv
MODFICADO ----- ./cleaned/MICRODATOS_ACC_VICT_2013/TABLA_ACCVICT_2013.csv
MODFICADO ----- ./cleaned/MICRODATOS_ACC_VICT_2014/TABLA_ACCVICT_2014.csv
MODFICADO ----- ./cleaned/MICRODATOS_ACC_VICT_2015/TABLA_ACCVICT_2015.csv
CREADA VISTA CON DATOS DE ACCIDENTES Y VISTIMAS DE 2008 A 2015


###   2. VEHICULOS

In [33]:
#Cabecera del fichero de Vehiculos
Header_Vehic=Headers_List[2]
Header_Vehic

,Vehic_2008,Vehic_2015
0,ID_ACCIDENTE,ID_ACCIDENTE
1,ID_VEHICULO,ID_VEHICULO
2,ANIO_MATRICULA_VEHICULO,ANIO_MATRICULA_VEHICULO
3,MES_MATRICULA_VEHICULO,MES_MATRICULA_VEHICULO
4,TIPO_VEHICULO,TIPO_VEHICULO
5,ESTADO_VEHICULO,NaN
6,NUMERO_OCUPANTES_VEH,NUMERO_OCUPANTES_VEH
7,MERCANCIAS_PELIGROSAS,MERCANCIAS_PELIGROSAS
8,VEHICULO_INCENDIADO,VEHICULO_INCENDIADO
9,ANIO,ANIO


In [34]:
#Matego solo las columnas inguales y actualizo mi diccionario con las cabeceras que utilizare en el proceso de carga

Header_Vehic=Header_Vehic[Header_Vehic['Vehic_2008']==Header_Vehic['Vehic_2015']]
Header_Vehic=Header_Vehic['Vehic_2008'].tolist()
dic_all_headers={'Vehic':Header_Vehic}
dic_all_headers

#Genero un df vacio donde concatenare todos los años para ver la cantidad de valores de que 
#dispongo por cada columna

df_all_years=pd.DataFrame(index=None, columns=dic_all_headers['Vehic'])

#Cargo cada csv y concateno
for file in dic_all_files['Vehic']:
    df_vehic=pd.read_csv(file,delimiter=';',usecols=dic_all_headers['Vehic'],encoding="ISO-8859-1")
    #relleno los nulos con 999
    df_vehic.fillna(999,inplace=True)
    df_all_years=pd.concat([df_all_years,df_vehic],ignore_index=True)
    
#muestreo los datos
df_all_years.head(3)
df_all_years.tail(3)
df_all_years.sample(3)

#De las columnas del data frame final, que contiene todos los años, 
#examino los distintos valores de las columnas, para tratar de decidir 
#que campos conservare para el analisis.
for col in df_all_years.columns:
    #no saco los distinos valores para los id_accidente
    if col !='ID_ACCIDENTE':
        df_all_years[col].value_counts(dropna=False)

#Una vez examindas los resultado anteriores, decido descartar los siguientes campos de la vista:
df_all_years.drop(['MERCANCIAS_PELIGROSAS','VEHICULO_INCENDIADO'], axis=1, inplace=True)

#Actualizo mi diccionario con las cabeceras que utilizare en el proceso de carga

        
#------------------SOBREECRITURA DE LOS DATOS APLICANDO MODIFICACIONES--------------
#Decido eliminor las siguientes columnas, por el alto numero de registro desconocidos y 
#porque considero que no me aportan valor:

#MERCANCIAS_PELIGROSAS
#VEHICULO_INCENDIADO
for file in dic_all_files['Vehic']:
    df_vehic=pd.read_csv(file,delimiter=';',usecols=dic_all_headers['Vehic'],encoding="ISO-8859-1")
    #relleno los nulos con 999
    df_vehic.fillna(999,inplace=True)
    df_vehic.drop(['MERCANCIAS_PELIGROSAS','VEHICULO_INCENDIADO'],axis=1,inplace=True)
    df_vehic.to_csv(file,index=False,sep=";",encoding="ISO-8859-1")
    print("MODFICADO ----- "+file)

print("CREADA VISTA CON DATOS DE VEHICULOS DE 2008 A 2015 EN ./CLEANED/----")
df_all_years.to_csv("./cleaned/view_vehic_08_15.csv",index=False,sep=";",encoding="ISO-8859-1")

########   FIN DEL PROCESAMIENTO DE LOS DATASET DE VEHICULOS #########


{'Vehic': ['ID_ACCIDENTE',
  'ID_VEHICULO',
  'ANIO_MATRICULA_VEHICULO',
  'MES_MATRICULA_VEHICULO',
  'TIPO_VEHICULO',
  'NUMERO_OCUPANTES_VEH',
  'MERCANCIAS_PELIGROSAS',
  'VEHICULO_INCENDIADO',
  'ANIO']}

,ID_ACCIDENTE,ID_VEHICULO,ANIO_MATRICULA_VEHICULO,MES_MATRICULA_VEHICULO,TIPO_VEHICULO,NUMERO_OCUPANTES_VEH,MERCANCIAS_PELIGROSAS,VEHICULO_INCENDIADO,ANIO
0,2.008410e+12,2.0,9999.0,99.0,22.0,1.0,N,N,2008.0
1,2.008520e+12,2.0,1991.0,99.0,22.0,1.0,N,N,2008.0
2,2.009460e+12,3.0,2004.0,1.0,22.0,1.0,N,N,2008.0


,ID_ACCIDENTE,ID_VEHICULO,ANIO_MATRICULA_VEHICULO,MES_MATRICULA_VEHICULO,TIPO_VEHICULO,NUMERO_OCUPANTES_VEH,MERCANCIAS_PELIGROSAS,VEHICULO_INCENDIADO,ANIO
1239590,2.016990e+12,1.0,2011.0,1.0,6.0,1.0,0,999,2015.0
1239591,2.016990e+12,1.0,9999.0,99.0,6.0,1.0,0,999,2015.0
1239592,2.016990e+12,2.0,2011.0,1.0,6.0,4.0,0,999,2015.0


,ID_ACCIDENTE,ID_VEHICULO,ANIO_MATRICULA_VEHICULO,MES_MATRICULA_VEHICULO,TIPO_VEHICULO,NUMERO_OCUPANTES_VEH,MERCANCIAS_PELIGROSAS,VEHICULO_INCENDIADO,ANIO
569548,2.011430e+12,2.0,9999.0,99.0,6.0,1.0,0,999,2011.0
68155,2.008030e+12,1.0,1996.0,9.0,22.0,1.0,N,N,2008.0
742214,2.012990e+12,1.0,2006.0,1.0,5.0,999.0,0,999,2012.0


1.0     710054
2.0     438466
3.0      64903
4.0      16647
5.0       5005
6.0       1931
7.0        868
8.0        443
9.0        249
10.0       158
11.0       120
12.0        86
13.0        66
14.0        52
16.0        41
15.0        39
18.0        32
19.0        29
20.0        29
27.0        26
21.0        26
17.0        25
22.0        23
23.0        22
24.0        17
25.0        15
26.0        15
28.0        14
29.0        13
30.0        13
         ...  
57.0         1
59.0         1
88.0         1
60.0         1
87.0         1
86.0         1
85.0         1
84.0         1
83.0         1
82.0         1
81.0         1
90.0         1
79.0         1
76.0         1
75.0         1
80.0         1
73.0         1
72.0         1
71.0         1
70.0         1
69.0         1
68.0         1
67.0         1
66.0         1
65.0         1
91.0         1
63.0         1
62.0         1
61.0         1
74.0         1
Name: ID_VEHICULO, dtype: int64

...

1.0      832129
2.0      188288
999.0    114350
3.0       48701
4.0       24477
0.0       18881
5.0        9476
6.0         969
7.0         455
8.0         251
9.0         212
10.0        181
20.0        133
15.0        115
11.0         77
30.0         73
12.0         72
25.0         43
40.0         42
16.0         42
14.0         42
13.0         41
18.0         37
50.0         36
21.0         32
22.0         25
17.0         24
23.0         24
35.0         23
19.0         23
          ...  
56.0         12
31.0         11
45.0         10
28.0         10
48.0          9
39.0          9
42.0          9
43.0          9
55.0          8
54.0          8
49.0          8
57.0          7
34.0          7
37.0          7
44.0          7
46.0          7
38.0          6
52.0          6
47.0          5
51.0          5
53.0          5
61.0          4
33.0          4
58.0          3
66.0          2
59.0          2
70.0          2
67.0          1
68.0          1
63.0          1
Name: NUMERO_OCUPANTES_V

0      772212
N      258104
999    208699
1         306
4         135
3         104
2          23
1          10
Name: MERCANCIAS_PELIGROSAS, dtype: int64

999    824532
N      414254
1.0       445
S         362
Name: VEHICULO_INCENDIADO, dtype: int64

2015.0    170749
2008.0    163785
2014.0    158835
2013.0    155004
2009.0    154013
2010.0    149277
2011.0    144274
2012.0    143656
Name: ANIO, dtype: int64

MODFICADO ----- ./cleaned/MICRODATOS_ACC_VICT_2008/TABLA_VEHIC_24H_2008.csv
MODFICADO ----- ./cleaned/MICRODATOS_ACC_VICT_2009/TABLA_VEHIC_24H_2009.csv
MODFICADO ----- ./cleaned/MICRODATOS_ACC_VICT_2010/TABLA_VEHIC_24H_2010.csv
MODFICADO ----- ./cleaned/MICRODATOS_ACC_VICT_2011/TABLA_VEHIC_2011.csv
MODFICADO ----- ./cleaned/MICRODATOS_ACC_VICT_2012/TABLA_VEHIC_2012.csv
MODFICADO ----- ./cleaned/MICRODATOS_ACC_VICT_2013/TABLA_VEHIC_2013.csv
MODFICADO ----- ./cleaned/MICRODATOS_ACC_VICT_2014/TABLA_VEHIC_2014.csv
MODFICADO ----- ./cleaned/MICRODATOS_ACC_VICT_2015/TABLA_VEHIC_2015.csv
CREADA VISTA CON DATOS DE VEHICULOS DE 2008 A 2015 EN ./CLEANED/----


In [44]:
df_vehic=pd.read_csv('./cleaned/MICRODATOS_ACC_VICT_2013/TABLA_VEHIC_2013.csv',delimiter=';',encoding="ISO-8859-1")
df_vehic.sample(5)

,ID_ACCIDENTE,ID_VEHICULO,ANIO_MATRICULA_VEHICULO,MES_MATRICULA_VEHICULO,TIPO_VEHICULO,NUMERO_OCUPANTES_VEH,ANIO
75174,2013250021329,3,9999,99,6,1,2013
128583,2013480052241,1,2001,1,16,20,2013
37199,2013080012569,2,9999,99,4,2,2013
60240,2013170018555,1,1997,1,6,1,2013
141747,2013990007088,1,2006,1,6,999,2013


In [23]:
df_vehic

,ID_ACCIDENTE,ID_VEHICULO,ANIO_MATRICULA_VEHICULO,MES_MATRICULA_VEHICULO,TIPO_VEHICULO,NUMERO_OCUPANTES_VEH,ANIO
0,2008410002330,2,9999,99,22,1,2008
1,2008520000162,2,1991,99,22,1,2008
2,2009460000665,3,2004,1,22,1,2008
3,2008460007725,1,2003,1,22,5,2008
4,2008410000371,1,2006,7,22,1,2008
5,2008410002803,1,1998,3,11,1,2008
6,2008460007689,2,2000,2,22,1,2008
7,2008460003425,1,2001,9,22,2,2008
8,2008410003099,1,2004,3,2,1,2008
9,2008460008075,1,2001,7,22,1,2008


###   1. PERSONAS

### TRATAMIENTO DATA SET PERSONAS

#### Se observa una diferencia en cuanto a la codificacion en los datos de sexo
#### entre los datasets anteriores y posteriores a 2011:

#####    -Para años anteriores a 2011 la codificacion es V o H
#####    -Para años posteriores, aparece codificados como 1 o 2, por lo que voy a intentar averiguar cual corresponde a hombre y mujeres
    
#### Es logico pensar que en los años anteriores a 1960, por ejemplo, el numero de hombres conductores era considerablemente al de mujeres, por lo que voy agrupar por edades y año de permiso de circulacion


#### Repito el procedimiento anterior para los datos desde 2011 a 2015 grabando los xls modificados en la carpeta cleaned junto con los otros data sets de años 2008 a 2010

### Repito el procedimiento anterior para los datos desde 2011 a 2015
### grabando los xls modificados en la carpeta cleaned junto con los otros data set 
### de años 2008 a 2010

In [11]:
#Cabecera del fichero de Personas
Header_Pers=Headers_List[1]
Header_Pers

,Pers_2008,Pers_2015
0,ID_ACCIDENTE,ID_ACCIDENTE
1,ID_VEHICULO,ID_VEHICULO
2,ID_PERSONA,ID_PERSONA
3,ID_CONDUCTOR,ID_CONDUCTOR
4,ID_PASAJERO,ID_PASAJERO
5,ID_PEATON,ID_PEATON
6,EDAD,EDAD
7,SEXO,SEXO
8,ANIO_PERMISO,ANIO_PERMISO
9,COD_POSICION,NaN


In [12]:
Header_Pers=Header_Pers[Header_Pers['Pers_2008']==Header_Pers['Pers_2015']]
Header_Pers=Header_Pers['Pers_2008'].tolist()
dic_all_headers={'Pers':Header_Pers}
dic_all_headers

df_all_years=pd.DataFrame(index=None, columns=dic_all_headers['Pers'])

#Cargo cada csv y concateno
for file in dic_all_files['Pers']:
    df_pers=pd.read_csv(file,delimiter=';',usecols=dic_all_headers['Pers'],encoding="ISO-8859-1")
    #relleno los nulos con 999
    df_pers.fillna(999,inplace=True)
    df_all_years=pd.concat([df_all_years,df_pers],ignore_index=True)
    
#muestreo los datos
df_all_years.head(3)
df_all_years.tail(3)
df_all_years.sample(3)

#De las columnas del data frame final, que contiene todos los años, 
#examino los distintos valores de las columnas, para tratar de decidir 
#que campos conservare para el analisis.
for col in df_all_years.columns:
    #no saco los distinos valores para los id_accidente
    if col !='ID_ACCIDENTE':
        df_all_years[col].value_counts(dropna=False)

#Modifico el sexo y el año de permiso en cada fichero origen

for file in range(3,8):
    #leo fichero
    df_pers_sexo=pd.read_csv(dic_all_files['Pers'][file],delimiter=";",usecols=dic_all_headers['Pers'],encoding="ISO-8859-1")
    #Para sexo desconocido relleno aleatoriamente con 1 o 2
    df_pers_sexo[df_pers_sexo['SEXO']==999]=np.random.randint(1,2)

    #selecciono las personas que son conductores ('ID_PERSONA'=1) y los años desconocidos (9999) que quiero sutituir
    pers_grp=df_pers_sexo[(df_pers_sexo['ID_PERSONA']==1) & (df_pers_sexo['ANIO_PERMISO']!=9999)].groupby(['EDAD','SEXO'],as_index=False)['ANIO_PERMISO'].agg({'NUM_CONDUCTORES':'count','MEDIA_ANTIG_CARNET':'mean'})
    #conversion de a entero
    pers_grp['MEDIA_ANTIG_CARNET']=pers_grp['MEDIA_ANTIG_CARNET'].round(0)
    pers_grp['MEDIA_ANTIG_CARNET']=pers_grp['MEDIA_ANTIG_CARNET'].astype(int)

    #cruzo el df original por edad y sexo con el df con las medias
    merged=pd.merge(df_pers_sexo, pers_grp, how='left',on=['EDAD','SEXO'],suffixes=['_left', '_right'])
    merged.head()

    #1. sutituyo los 1 por V y los 2 por H
    merged.ix[(merged['SEXO']==1), 'SEXO'] ='V'
    merged.ix[(merged['SEXO']==2), 'SEXO'] ='M'
    #2. sustituyo los valores de años de permisos deconocidos
    merged.ix[((merged['ANIO_PERMISO']==9999) | (merged['ANIO_PERMISO']==0.0)) & (df_pers_sexo['ID_PERSONA']==1), 'ANIO_PERMISO'] =merged['MEDIA_ANTIG_CARNET']
    merged.head()
    #elimino del df las columnas con el numeor de conductores y la media de año de carnet
    merged.drop(['NUM_CONDUCTORES','MEDIA_ANTIG_CARNET'], axis=1, inplace=True)

    #si hubiese algun año NULL, lo relleno a 0.0
    merged['ANIO_PERMISO'].fillna(0.0,inplace=True)
    merged['ANIO_PERMISO']=merged['ANIO_PERMISO'].astype(int)

    #existen errores en los años de permiso, para conductores tenemos valores 0,1 y 999
    #muestro los diferentes valores de año de permiso
    pd.value_counts(merged['ANIO_PERMISO'])
    #elimino esos registros
    merged=merged[(merged['ANIO_PERMISO'].isin([0,1,999])) & merged['ID_PERSONA']!=1]
    #guardo el fichero
    merged.to_csv(dic_all_files['Pers'][file],index=False,sep=";",encoding="ISO-8859-1")
    print (dic_all_files['Pers'][file]+" --- modificado"+"\n")


#Genero la vista con los datos de todos los años
df_all_years=pd.DataFrame(index=None, columns=dic_all_headers['Pers'])

#Cargo de nuevo ya que he modificado las tablas que componen la vista y la genero de nuevo
for file in dic_all_files['Pers']:
    df_pers=pd.read_csv(file,delimiter=';',usecols=dic_all_headers['Pers'],encoding="ISO-8859-1")
    #relleno los nulos con 999
    df_pers.fillna(999,inplace=True)
    df_all_years=pd.concat([df_all_years,df_pers],ignore_index=True)
    
print("CREADA VISTA CON DATOS DE VEHICULOS DE 2008 A 2015 EN ./CLEANED/----")
df_all_years.to_csv("./cleaned/view_pers_08_15.csv",index=False,sep=";",encoding="ISO-8859-1")

{'Pers': ['ID_ACCIDENTE',
  'ID_VEHICULO',
  'ID_PERSONA',
  'ID_CONDUCTOR',
  'ID_PASAJERO',
  'ID_PEATON',
  'EDAD',
  'SEXO',
  'ANIO_PERMISO',
  'MANIOBRAS',
  'INFRACC_VELOCIDAD',
  'INFRACC_PEATON',
  'ANIO']}

,ID_ACCIDENTE,ID_VEHICULO,ID_PERSONA,ID_CONDUCTOR,ID_PASAJERO,ID_PEATON,EDAD,SEXO,ANIO_PERMISO,MANIOBRAS,INFRACC_VELOCIDAD,INFRACC_PEATON,ANIO
0,"2,00801E+12",1.0,2.0,999.0,1.0,999.0,32.0,M,9999.0,999.0,999.0,999.0,2008.0
1,"2,00801E+12",2.0,2.0,999.0,1.0,999.0,60.0,M,9999.0,999.0,999.0,999.0,2008.0
2,"2,00801E+12",2.0,2.0,999.0,2.0,999.0,23.0,V,9999.0,999.0,999.0,999.0,2008.0


,ID_ACCIDENTE,ID_VEHICULO,ID_PERSONA,ID_CONDUCTOR,ID_PASAJERO,ID_PEATON,EDAD,SEXO,ANIO_PERMISO,MANIOBRAS,INFRACC_VELOCIDAD,INFRACC_PEATON,ANIO
1724733,2016990000358,999.0,3.0,999.0,999.0,1.0,31.0,1,999.0,999.0,999.0,2.0,2015.0
1724734,2016990000448,999.0,3.0,999.0,999.0,1.0,7.0,1,999.0,999.0,999.0,2.0,2015.0
1724735,2016990000876,999.0,3.0,999.0,999.0,2.0,7.0,2,999.0,999.0,999.0,2.0,2015.0


,ID_ACCIDENTE,ID_VEHICULO,ID_PERSONA,ID_CONDUCTOR,ID_PASAJERO,ID_PEATON,EDAD,SEXO,ANIO_PERMISO,MANIOBRAS,INFRACC_VELOCIDAD,INFRACC_PEATON,ANIO
327777,"2,00928E+12",1.0,1.0,1.0,999.0,999.0,51.0,V,2004.0,11.0,4.0,999.0,2009.0
367246,"2,00941E+12",1.0,2.0,999.0,1.0,999.0,23.0,V,9999.0,999.0,999.0,999.0,2009.0
1304763,2014080014553,1.0,1.0,1.0,999.0,999.0,32.0,1,2004.0,52.0,3.0,999.0,2014.0


1.0      944267
2.0      573607
999.0     97957
3.0       78297
4.0       19402
5.0        5733
6.0        2203
7.0        1023
8.0         503
9.0         269
10.0        188
11.0        149
13.0        102
12.0         99
14.0         67
16.0         60
15.0         54
19.0         50
20.0         48
21.0         46
22.0         44
43.0         44
18.0         42
27.0         38
17.0         37
23.0         37
24.0         29
25.0         29
26.0         26
38.0         23
          ...  
57.0          1
56.0          1
91.0          1
58.0          1
73.0          1
89.0          1
88.0          1
87.0          1
85.0          1
84.0          1
83.0          1
82.0          1
81.0          1
92.0          1
76.0          1
75.0          1
71.0          1
59.0          1
70.0          1
69.0          1
68.0          1
67.0          1
66.0          1
65.0          1
93.0          1
63.0          1
64.0          1
62.0          1
61.0          1
60.0          1
Name: ID_VEHICULO, dtype

...

./cleaned/MICRODATOS_ACC_VICT_2014/TABLA_PERS_2014.csv --- modificado



,ID_ACCIDENTE,ID_VEHICULO,ID_PERSONA,ID_CONDUCTOR,ID_PASAJERO,ID_PEATON,EDAD,SEXO,ANIO_PERMISO,MANIOBRAS,INFRACC_VELOCIDAD,INFRACC_PEATON,ANIO,NUM_CONDUCTORES,MEDIA_ANTIG_CARNET
0,2015010050001,1.0,1,1.0,NaN,NaN,42,1,9999.0,1.0,3.0,NaN,2015,2268.0,1997.0
1,2015010050002,1.0,1,1.0,NaN,NaN,44,1,9999.0,1.0,3.0,NaN,2015,2188.0,1995.0
2,2015010050002,2.0,1,2.0,NaN,NaN,32,1,9999.0,1.0,3.0,NaN,2015,2191.0,2005.0
3,2015010050003,1.0,1,1.0,NaN,NaN,50,1,9999.0,1.0,3.0,NaN,2015,1793.0,1990.0
4,2015010050003,2.0,1,2.0,NaN,NaN,52,1,9999.0,1.0,3.0,NaN,2015,1709.0,1988.0


,ID_ACCIDENTE,ID_VEHICULO,ID_PERSONA,ID_CONDUCTOR,ID_PASAJERO,ID_PEATON,EDAD,SEXO,ANIO_PERMISO,MANIOBRAS,INFRACC_VELOCIDAD,INFRACC_PEATON,ANIO,NUM_CONDUCTORES,MEDIA_ANTIG_CARNET
0,2015010050001,1.0,1,1.0,NaN,NaN,42,V,1997.0,1.0,3.0,NaN,2015,2268.0,1997.0
1,2015010050002,1.0,1,1.0,NaN,NaN,44,V,1995.0,1.0,3.0,NaN,2015,2188.0,1995.0
2,2015010050002,2.0,1,2.0,NaN,NaN,32,V,2005.0,1.0,3.0,NaN,2015,2191.0,2005.0
3,2015010050003,1.0,1,1.0,NaN,NaN,50,V,1990.0,1.0,3.0,NaN,2015,1793.0,1990.0
4,2015010050003,2.0,1,2.0,NaN,NaN,52,V,1988.0,1.0,3.0,NaN,2015,1709.0,1988.0


0       73898
2014     6678
2007     6667
2008     6622
2006     6309
2009     5716
2013     5477
2010     5294
2011     5118
2015     4944
2004     4933
2012     4829
2005     4768
1998     4395
1992     4354
2003     4231
1996     4092
2002     3878
1997     3764
1995     3592
1990     3580
1993     3565
1994     3504
2000     3296
2001     3263
1989     3236
1999     3222
1991     3176
1        3162
1988     3093
        ...  
1981     1294
1974     1260
1972     1138
1971     1038
1969      872
1970      788
1967      670
1968      584
1966      562
1965      516
1964      437
1963      357
1962      303
1961      190
1960      122
1959      106
1958       90
1957       64
1956       42
1955       25
1954       20
1953       15
1952        8
1949        5
1948        4
1951        3
1950        3
1947        3
1945        3
1942        1
Name: ANIO_PERMISO, dtype: int64

./cleaned/MICRODATOS_ACC_VICT_2015/TABLA_PERS_2015.csv --- modificado

CREADA VISTA CON DATOS DE VEHICULOS DE 2008 A 2015 EN ./CLEANED/----


### ESTAS SERIAN LA COLUMNAS FINALES DE LOS 3 DATASETS

In [13]:
dic_all_headers={'Accvict':Header_Accvict,'Pers':Header_Pers,'Vehic':Header_Vehic}
dic_all_headers


{'Accvict': ['ID_ACCIDENTE',
  'ANIO',
  'MES',
  'HORA',
  'DIASEMANA',
  'PROVINCIA',
  'COMUNIDAD_AUTONOMA',
  'TOT_VICTIMAS',
  'TOT_MUERTOS',
  'TOT_HERIDOS_GRAVES',
  'TOT_HERIDOS_LEVES',
  'TOT_VEHICULOS_IMPLICADOS',
  'ZONA',
  'ZONA_AGRUPADA',
  'RED_CARRETERA',
  'TIPO_VIA',
  'SUPERFICIE_CALZADA',
  'LUMINOSIDAD',
  'FACTORES_ATMOSFERICOS',
  'TIPO_ACCIDENTE'],
 'Pers': ['ID_ACCIDENTE',
  'ID_VEHICULO',
  'ID_PERSONA',
  'ID_CONDUCTOR',
  'ID_PASAJERO',
  'ID_PEATON',
  'EDAD',
  'SEXO',
  'ANIO_PERMISO',
  'MANIOBRAS',
  'INFRACC_VELOCIDAD',
  'INFRACC_PEATON',
  'ANIO'],
 'Vehic': ['ID_ACCIDENTE',
  'ID_VEHICULO',
  'ANIO_MATRICULA_VEHICULO',
  'MES_MATRICULA_VEHICULO',
  'TIPO_VEHICULO',
  'NUMERO_OCUPANTES_VEH',
  'MERCANCIAS_PELIGROSAS',
  'VEHICULO_INCENDIADO',
  'ANIO']}

### -------------------------------------------------------------------------------------------------------
### PROCEDO A CARGAR EN DATAFRAMES DE CADA UNO DE LOS FICHEROS DEL DICCIONARIO
### -------------------------------------------------------------------------------------------------------

In [14]:
#guardo las claves de cada tipo de fichero
list_keys_dic=dic_all_files.keys()
list_keys_dic
#Actualizo el diccionario con la rutas a los ficheros de trabajo
dic_all_files['Accvict'].append('./cleaned/view_accvict_08_15.csv')
dic_all_files['Pers'].append('./cleaned/view_pers_08_15.csv')
dic_all_files['Vehic'].append('./cleaned/view_vehic_08_15.csv')
dic_all_files

dict_keys(['Accvict', 'Pers', 'Vehic'])

{'Accvict': ['./cleaned/MICRODATOS_ACC_VICT_2008/TABLA_ACCVICT_24H_2008.csv',
  './cleaned/MICRODATOS_ACC_VICT_2009/TABLA_ACCVICT_24H_2009.csv',
  './cleaned/MICRODATOS_ACC_VICT_2010/TABLA_ACCVICT_24H_2010.csv',
  './cleaned/MICRODATOS_ACC_VICT_2011/TABLA_ACCVICT_2011.csv',
  './cleaned/MICRODATOS_ACC_VICT_2012/TABLA_ACCVICT_2012.csv',
  './cleaned/MICRODATOS_ACC_VICT_2013/TABLA_ACCVICT_2013.csv',
  './cleaned/MICRODATOS_ACC_VICT_2014/TABLA_ACCVICT_2014.csv',
  './cleaned/MICRODATOS_ACC_VICT_2015/TABLA_ACCVICT_2015.csv',
  './cleaned/view_accvict_08_15.csv'],
 'Pers': ['./cleaned/MICRODATOS_ACC_VICT_2008/TABLA_PERS_24H_2008.csv',
  './cleaned/MICRODATOS_ACC_VICT_2009/TABLA_PERS_24H_2009.csv',
  './cleaned/MICRODATOS_ACC_VICT_2010/TABLA_PERS_24H_2010.csv',
  './cleaned/MICRODATOS_ACC_VICT_2011/TABLA_PERS_2011.csv',
  './cleaned/MICRODATOS_ACC_VICT_2012/TABLA_PERS_2012.csv',
  './cleaned/MICRODATOS_ACC_VICT_2013/TABLA_PERS_2013.csv',
  './cleaned/MICRODATOS_ACC_VICT_2014/TABLA_PERS_2014.

In [45]:
dic_all_files['Accvict'][8]


IndexError: list index out of range

In [15]:
#Creo un dataframe vacio para posteriormente rellenarlo con los años 2001 al 2015
#Recorriendo por cada key los n values del diccionario

dic_total={}
df_total=pd.DataFrame()

for key in list_keys_dic:
    print ("#### "+key+" ####")
    for file in dic_all_files[key]:
        #meto en df cada file quedandome solo con las columnas seleccionadas anteriormente
        df_year=pd.read_csv(file,delimiter=";",encoding="ISO-8859-1",usecols=dic_all_headers[key])
        df_year=df_year.fillna(0)
        print("Cargando fichero-->"+file+"\n\t"+"Registros= "+str(df_year.shape[0])+"\n\t"+"Columnas= "+str(df_year.shape[1]))
        #df_total=df_total.append(df_year)
        df_total=pd.concat([df_total,df_year],ignore_index=True)
    dic_total[key] = df_total
print ("\n\t\t"+"##### CARGADOS EN DF_TOTAL TODOS LOS PERIODOS ########"+"\t\t")
df_total.shape

#### Accvict ####
Cargando fichero-->./cleaned/MICRODATOS_ACC_VICT_2008/TABLA_ACCVICT_24H_2008.csv
	Registros= 93161
	Columnas= 27
Cargando fichero-->./cleaned/MICRODATOS_ACC_VICT_2009/TABLA_ACCVICT_24H_2009.csv
	Registros= 88251
	Columnas= 27
Cargando fichero-->./cleaned/MICRODATOS_ACC_VICT_2010/TABLA_ACCVICT_24H_2010.csv
	Registros= 85503
	Columnas= 27
Cargando fichero-->./cleaned/MICRODATOS_ACC_VICT_2011/TABLA_ACCVICT_2011.csv
	Registros= 83027
	Columnas= 27
Cargando fichero-->./cleaned/MICRODATOS_ACC_VICT_2012/TABLA_ACCVICT_2012.csv
	Registros= 83115
	Columnas= 27
Cargando fichero-->./cleaned/MICRODATOS_ACC_VICT_2013/TABLA_ACCVICT_2013.csv
	Registros= 89519
	Columnas= 27
Cargando fichero-->./cleaned/MICRODATOS_ACC_VICT_2014/TABLA_ACCVICT_2014.csv
	Registros= 91570
	Columnas= 27
Cargando fichero-->./cleaned/MICRODATOS_ACC_VICT_2015/TABLA_ACCVICT_2015.csv
	Registros= 97756
	Columnas= 27
#### Pers ####
Cargando fichero-->./cleaned/MICRODATOS_ACC_VICT_2008/TABLA_PERS_24H_2008.csv
	Regi

(3585277, 44)

#### AHORA TENGO DE DATA FRAMES CON TODOS LOS AÑOS DESDE 2008 HASTA 2015

In [92]:
##para categorizar
#dic_total['Accvict']['HORA']=dic_total['Accvict']['HORA'].astype('category')
#dic_total['Accvict']['HORA'].describe()
#este funciona
dic_total['Accvict']['HORA']= pd.Series(pd.Categorical(dic_total['Accvict']['HORA'], categories=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]))
dic_total['Accvict']['HORA'].cat.rename_categories(['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23'])
dic_total['Accvict']['HORA'].cat.codes

count     711902
unique        24
top           14
freq       50900
Name: HORA, dtype: int64

In [22]:
#Extraigo los datos cuantitativos de victimas por
# 1. Año-mes-dia de la semana
# 2. Localizacion geografica

accvict_geo=pd.get_dummies(dic_total['Accvict'], prefix=['CA','PROV'], columns=['COMUNIDAD_AUTONOMA','PROVINCIA'])


In [23]:
dic_total['Accvict']

,ID_ACCIDENTE,ANIO,MES,HORA,DIASEMANA,PROVINCIA,COMUNIDAD_AUTONOMA,ISLA,MUNICIPIO,TOT_VICTIMAS,...,RED_CARRETERA,TIPO_VIA,TRAZADO_NO_INTERSEC,TIPO_INTERSEC,SUPERFICIE_CALZADA,LUMINOSIDAD,FACTORES_ATMOSFERICOS,VISIBILIDAD_RESTRINGIDA,ACERAS,TIPO_ACCIDENTE
0,2008480000372,2008,4,07:00,5,48,17,0.0,0.0,4,...,3,2,1.0,0.0,2,3,1,8,N,71
1,2008250021218,2008,3,14:00,7,25,9,0.0,0.0,1,...,2,5,2.0,0.0,1,1,9,8,N,22
2,2008430024095,2008,10,12:00,7,43,9,0.0,0.0,1,...,4,5,1.0,0.0,1,1,1,0,N,68
3,2008430022973,2008,4,18:00,2,43,9,0.0,0.0,1,...,4,5,0.0,3.0,1,1,1,0,N,14
4,2008250021457,2008,5,19:00,5,25,9,0.0,0.0,1,...,4,9,1.0,0.0,9,1,9,8,N,66
5,2008250022083,2008,12,14:00,1,25,9,0.0,0.0,3,...,2,5,2.0,0.0,1,1,1,8,N,67
6,2008430024521,2008,12,20:00,2,43,9,0.0,0.0,1,...,4,9,1.0,0.0,1,5,1,0,N,12
7,2008480000579,2008,6,19:00,5,48,17,0.0,0.0,2,...,5,1,1.0,0.0,1,1,1,8,N,13
8,2008430022598,2008,3,02:00,7,43,9,0.0,0.0,2,...,4,5,0.0,1.0,1,5,1,0,N,68
9,2008080018378,2008,12,13:00,2,8,9,0.0,0.0,1,...,4,5,0.0,3.0,9,1,1,0,N,41


In [24]:
df = pd.DataFrame({'A': ['x', 'y', 'x'], 'B': ['z', 'u', 'z'],
                  'C': [1,2,3],
                  'D':['j', 'l', 'j']})

In [25]:
df

,A,B,C,D
0,x,z,1,j
1,y,u,2,l
2,x,z,3,j


In [26]:
df_1= pd.get_dummies(df)
df_1

,C,A_x,A_y,B_u,B_z,D_j,D_l
0,1,1,0,0,1,1,0
1,2,0,1,1,0,0,1
2,3,1,0,0,1,1,0


In [27]:
df_2=pd.get_dummies(df, prefix=['DDDD','DDD','D'], columns=['A','B','C'])
df_2

,D,DDDD_x,DDDD_y,DDD_u,DDD_z,D_1,D_2,D_3
0,j,1,0,0,1,1,0,0
1,l,0,1,1,0,0,1,0
2,j,1,0,0,1,0,0,1


In [28]:
  df_year=pd.read_csv(files_list[file],delimiter=";",encoding="ISO-8859-1",converters={'COD_MUNICIPIO': str, 'MUNICIPIO': str})

NameError: name 'files_list' is not defined


### LECTURA DE LOS DATASET DE VEHICULOS PARA LOS AÑOS 2011-2015

In [ ]:
df_year.shape[0]

In [187]:
#Creo un dataframe vacio para posteriormente rellenarlo con los años 2001 al 2015
df_accvict_08_15 = pd.DataFrame()
dict={}
for key in list_keys_dic:
    for file in range(0,len(files_list)):
        df_year=pd.read_csv(files_list[file],delimiter=";",encoding="ISO-8859-1")
        df_year.fillna(0)
        dict[tipology]=files_list[file]
        print("Cargando fichero-->"+files_list[file]+"\n\t"+"Registros= "+str(df_year.shape[0])+"\n\t"+"Columnas= "+str(df_year.shape[1]))
        if tipology=='list_accvict'
        %(tipology)+_08_15=df_accvict_08_15.append(df_year)


Cargando fichero-->./MICRODATOS_ACC_VICT_2008/TABLA_ACCVICT_24H_2008.csv
	Registros= 93161
	Columnas= 32
Cargando fichero-->./MICRODATOS_ACC_VICT_2009/TABLA_ACCVICT_24H_2009.csv
	Registros= 88251
	Columnas= 32
Cargando fichero-->./MICRODATOS_ACC_VICT_2010/TABLA_ACCVICT_24H_2010.csv
	Registros= 85503
	Columnas= 32
Cargando fichero-->./MICRODATOS_ACC_VICT_2011/TABLA_ACCVICT_2011.csv
	Registros= 83027
	Columnas= 39
Cargando fichero-->./MICRODATOS_ACC_VICT_2012/TABLA_ACCVICT_2012.csv
	Registros= 83115
	Columnas= 39
Cargando fichero-->./MICRODATOS_ACC_VICT_2013/TABLA_ACCVICT_2013.csv
	Registros= 89519
	Columnas= 39
Cargando fichero-->./MICRODATOS_ACC_VICT_2014/TABLA_ACCVICT_2014.csv
	Registros= 91570
	Columnas= 39
Cargando fichero-->./MICRODATOS_ACC_VICT_2015/TABLA_ACCVICT_2015.csv
	Registros= 97756
	Columnas= 39


In [189]:
j=df_accvict_08_15.groupby(['ANIO']).size()


In [191]:
type(j)

pandas.core.series.Series

In [ ]:
df_accvict_08_15.groupby(['ANIO']).agg(['mean', 'count'])

In [176]:
print(df_accvict_08_15.head())
#df1=df[['ANIO','TOT_VICTIMAS']]


  ACERAS  ACOND_CALZADA  ANIO CARRETERA  COD_MUNICIPIO  COMUNIDAD_AUTONOMA  \
0      N            0.0  2008    BI-631            NaN                  17   
1      N            0.0  2008      C-12            NaN                   9   
2      N            0.0  2008   TV-7331            NaN                   9   
3      N            0.0  2008     N-340            NaN                   9   
4      N            0.0  2008     CR-26            NaN                   9   

   DENSIDAD_CIRCULACION  DIASEMANA  FACTORES_ATMOSFERICOS   HORA  \
0                   1.0          5                      1  07:00   
1                   1.0          7                      9  14:00   
2                   0.0          7                      1  12:00   
3                   0.0          2                      1  18:00   
4                   1.0          5                      9  19:00   

       ...        TOT_HERIDOS_LEVES30D  TOT_MUERTOS  TOT_MUERTOS30D  \
0      ...                         NaN            0

In [174]:
dict

{0: './MICRODATOS_ACC_VICT_2008/TABLA_ACCVICT_24H_2008.csv',
 1: './MICRODATOS_ACC_VICT_2009/TABLA_ACCVICT_24H_2009.csv',
 2: './MICRODATOS_ACC_VICT_2010/TABLA_ACCVICT_24H_2010.csv',
 3: './MICRODATOS_ACC_VICT_2011/TABLA_ACCVICT_2011.csv',
 4: './MICRODATOS_ACC_VICT_2012/TABLA_ACCVICT_2012.csv',
 5: './MICRODATOS_ACC_VICT_2013/TABLA_ACCVICT_2013.csv',
 6: './MICRODATOS_ACC_VICT_2014/TABLA_ACCVICT_2014.csv',
 7: './MICRODATOS_ACC_VICT_2015/TABLA_ACCVICT_2015.csv'}

In [ ]:
threedee.scatter(df.ANIO, df["TOT_MUERTOS"], df['Volume'])